In [0]:
spark

Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
dbutils.library.installPyPI("great_expectations")

dbutils.library APIs are deprecated and will be removed in a future DBR release. You can use %pip and %conda commands to install notebook scoped python libraries. For more information see https://docs.databricks.com/libraries/notebooks-python-libraries.html.
PyPI package great_expectations has been installed already. The previously installed package is `great_expectations`. To resolve this issue, detach and re-attach the notebook to create a new environment or rename the package.
Out[17]: False

In [0]:
context.list_datasources()

Out[77]: [{'class_name': 'Datasource',
 'execution_engine': {'class_name': 'SparkDFExecutionEngine'},
 'module_name': 'great_expectations.datasource',
 'data_connectors': {'spark_runtime_connector1': {'class_name': 'RuntimeDataConnector1',
 'module_name': 'great_expectations.datasource.data_connector'}},
 'name': 'spark_data_source3'}]

In [0]:
################################

In [0]:
from great_expectations.data_context.types.base import DatasourceConfig
from great_expectations.core.batch import BatchRequest
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context.types.base import DataContextConfig, DatasourceConfig, FilesystemStoreBackendDefaults
from great_expectations.data_context import BaseDataContext

# Load your data into a dataframe

file_location = "/FileStore/tables/Building_Permits.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

# Create a DataContext in code from a DataContextConfig with DatasourceConfig

my_spark_datasource_config = DatasourceConfig(
    class_name="Datasource",
    execution_engine={"class_name": "SparkDFExecutionEngine"},
    data_connectors={
      "my_runtime_data_connector_name": {
        "module_name": "great_expectations.datasource.data_connector",
        "class_name": "RuntimeDataConnector",
        "batch_identifiers": [
          "some_batch_identifier_so_this_can_work"
        ]
      }
    }
  )

project_config = DataContextConfig(
    datasources={"my_datasource_name": my_spark_datasource_config},
    store_backend_defaults=FilesystemStoreBackendDefaults(root_directory="/dbfs/FileStore/")
)

context = BaseDataContext(project_config=project_config)

# Create a RuntimeBatchRequest
batch_request = RuntimeBatchRequest(
    datasource_name="my_datasource_name",
    data_connector_name="my_runtime_data_connector_name",
    data_asset_name="my_data_asset_name",
runtime_parameters={
      "batch_data": df
    },
    batch_identifiers={
        "some_batch_identifier_so_this_can_work": "blah",
    }
)

# Create or load your Expectation Suite
# NOTE: You should either create or load, this try/except block is for convenience
from great_expectations.exceptions import DataContextError
try:
  suite = context.create_expectation_suite("my_suite_name1")
except DataContextError:
  suite = context.get_expectation_suite("my_suite_name1")

# Get a Validator
my_validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite=suite
)

# Add Expectations
my_validator.expect_table_row_count_to_equal(140)
my_validator.expect_column_values_to_not_be_null("Permit Number")
my_validator.expect_column_to_exist("Road Number")

# Save the Expectation Suite to the Expectation Store
my_validator.save_expectation_suite(discard_failed_expectations=False)

Calculating Metrics: 0%| | 0/8 [00:00<?, ?it/s]Calculating Metrics: 50%|█████ | 4/8 [00:00<00:00, 13.59it/s]Calculating Metrics: 88%|████████▊ | 7/8 [00:01<00:00, 4.45it/s]Calculating Metrics: 100%|██████████| 8/8 [00:01<00:00, 3.69it/s]Calculating Metrics: 100%|██████████| 8/8 [00:01<00:00, 4.32it/s]

In [0]:
context.get_expectation_suite("my_suite_name1")

Out[129]: {
 "data_asset_type": null,
 "meta": {
 "great_expectations_version": "0.13.32"
 },
 "expectation_suite_name": "my_suite_name1",
 "expectations": [
 {
 "kwargs": {
 "value": 140
 },
 "expectation_type": "expect_table_row_count_to_equal",
 "meta": {}
 },
 {
 "kwargs": {
 "column": "Permit Number"
 },
 "expectation_type": "expect_column_values_to_not_be_null",
 "meta": {}
 },
 {
 "kwargs": {
 "column": "Road Number"
 },
 "expectation_type": "expect_column_to_exist",
 "meta": {}
 }
 ]
}

In [0]:
from great_expectations.data_context import BaseDataContext

file_location = "/FileStore/tables/Building_Permits.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

# NOTE: project_config is a DataContextConfig set up as in the examples above.
context = BaseDataContext(project_config=data_context_config)
suite = context.get_expectation_suite("my_suite_name")



my_batch = context.get_batch({
  "dataset": df,
  "datasource": "my_spark_datasource",
}, context.get_expectation_suite("my_suite_name1"))

my_batch.validate()


Out[130]: {
 "evaluation_parameters": {},
 "meta": {
 "great_expectations_version": "0.13.32",
 "expectation_suite_name": "my_suite_name1",
 "run_id": {
 "run_time": "2021-09-04T12:24:57.091258+00:00",
 "run_name": null
 },
 "batch_kwargs": {
 "datasource": "my_spark_datasource",
 "SparkDFRef": true,
 "ge_batch_id": "1d9ed0f2-0d7b-11ec-84a2-00163eff7204"
 },
 "batch_markers": {
 "ge_load_time": "20210904T122457.072789Z"
 },
 "batch_parameters": null,
 "validation_time": "20210904T122457.091135Z",
 "expectation_suite_meta": {
 "great_expectations_version": "0.13.32"
 }
 },
 "results": [
 {
 "result": {
 "observed_value": 198900
 },
 "exception_info": {
 "raised_exception": false,
 "exception_message": null,
 "exception_traceback": null
 },
 "meta": {},
 "success": false,
 "expectation_config": {
 "kwargs": {
 "value": 140
 },
 "expectation_type": "expect_table_row_count_to_equal",
 "meta": {}
 }
 },
 {
 "result": {
 "element_count": 198900,
 "unexpected_count": 0,
 "unexpected_percent": 0.0,
 "unexpected_percent_total": 0.0,
 "partial_unexpected_list": []
 },
 "exception_info": {
 "raised_exception": false,
 "exception_message": null,
 "exception_traceback": null
 },
 "meta": {},
 "success": true,
 "expectation_config": {
 "kwargs": {
 "column": "Permit Number"
 },
 "expectation_type": "expect_column_values_to_not_be_null",
 "meta": {}
 }
 },
 {
 "result": {},
 "exception_info": {
 "raised_exception": false,
 "exception_message": null,
 "exception_traceback": null
 },
 "meta": {},
 "success": false,
 "expectation_config": {
 "kwargs": {
 "column": "Road Number"
 },
 "expectation_type": "expect_column_to_exist",
 "meta": {}
 }
 }
 ],
 "success": false,
 "statistics": {
 "evaluated_expectations": 3,
 "successful_expectations": 1,
 "unsuccessful_expectations": 2,
 "success_percent": 33.33333333333333
 }
}